In [1]:
# !pip install open_clip_torch -q

In [2]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
import open_clip
import glob
from collections import defaultdict
import shutil
from cfg import KEYFRAME_DIR, FEATURE_DIR, ZIP_PATH

# --- Configuration ---
base_dir = KEYFRAME_DIR # contains batch of videos frames: V1/V1_0001, V1/V1_0002, ...
output_dir = FEATURE_DIR # Save 1 .npy per image
zip_path = ZIP_PATH # one single .zip file

os.makedirs(output_dir, exist_ok=True)

# --- Load OpenCLIP ---
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, _, preprocess = open_clip.create_model_and_transforms(
    model_name='ViT-H-14-quickgelu',
    pretrained='dfn5b',
    device=device
)
model.eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-31): 32 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1280, out_features=5120, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=5120, out_features=1280, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=Tru

In [3]:
videos = glob.glob(os.path.join(base_dir,'*','*','*.jpg'))
if not videos:
    print("[ERROR] No videos found.")

print(f"[INFO] Found {len(videos)} frames to process.")

videos = sorted(videos)

[INFO] Found 17961 frames to process.


In [4]:
# check output dir and remove same name
existed = glob.glob(os.path.join(output_dir,'*','*','*.npy'))


keyframes_map = {os.path.splitext(os.path.basename(f))[0]: f for f in videos}

# Basenames from second list
features_set = {os.path.splitext(os.path.basename(f))[0] for f in existed}

filtered_first = [path for base, path in keyframes_map.items() if base not in features_set]

videos = sorted(filtered_first)

print(f"[INFO] Remaining {len(videos)} frames to process (removed existed features).")

[INFO] Remaining 15489 frames to process (removed existed features).


In [5]:
batch_size=4

folder_to_images = defaultdict(list)
for img_path in videos:
    folder_to_images[os.path.dirname(img_path)].append(img_path)

ii = 0
for folder_path, image_files in folder_to_images.items():
    print(f"{ii+1}/{len(folder_to_images)}")
    # Create matching output subfolder
    rel_path = os.path.relpath(folder_path, base_dir)
    save_folder = os.path.join(output_dir, rel_path)
    os.makedirs(save_folder, exist_ok=True)

    # Sort image paths for consistent processing
    image_files = sorted(image_files)
    # Process in batches
    for i in tqdm(range(0, len(image_files), batch_size), desc=f"Processing {rel_path}"):
        batch_files = image_files[i:i + batch_size]
        batch_images = []
        valid_files = []

        for img_path in batch_files:
            try:
                image = Image.open(img_path).convert('RGB')
                image_tensor = preprocess(image)
                batch_images.append(image_tensor)
                valid_files.append(img_path)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")

        if not batch_images:
            continue

        # Encode features
        input_tensor = torch.stack(batch_images).to(device)
        with torch.no_grad():
            features = model.encode_image(input_tensor)
            features = features / features.norm(dim=-1, keepdim=True)

        # Save each feature with same relative path
        for img_path, feat in zip(valid_files, features):
            rel_img_path = os.path.relpath(img_path, base_dir)
            save_path = os.path.join(output_dir, os.path.splitext(rel_img_path)[0] + '.npy')
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            np.save(save_path, feat.cpu().numpy())
        
    ii += 1

1/130


Processing L21/L21_V010: 100%|██████████| 57/57 [04:54<00:00,  5.17s/it]


2/130


Processing L21/L21_V011: 100%|██████████| 71/71 [05:50<00:00,  4.93s/it]


3/130


...

Processing L25/L25_V045: 100%|██████████| 27/27 [02:11<00:00,  4.87s/it]


129/130


Processing L25/L25_V046: 100%|██████████| 30/30 [02:29<00:00,  4.98s/it]


130/130


Processing L25/L25_V047: 100%|██████████| 22/22 [01:48<00:00,  4.95s/it]


In [6]:
shutil.make_archive(zip_path, 'zip', output_dir)
print(f"Zipped features saved to {zip_path}.zip")

Zipped features saved to /mnt/d/AI Challenge/Data/features/features.zip.zip
